<a href="https://colab.research.google.com/github/kamnsv/datawagon2/blob/main/DataWagon_2_3_WagonWizards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Импорт библиотек
import tensorflow as tf
import os